In [ ]:
# figure3G: rsc shared verus independent across natural images
# author: Amir Farzmahdi
# last update: May 23rd, 2024

In [ ]:
# library imports
import os
import pickle
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from PIL import Image
from scipy import stats

In [ ]:
# settings

n_theta = 9

simgs_idx = [12, 13, 19]
sz_color = ['#0072BD', '#D95319']
purple_rgb = [0.5, 0, 0.5]  
green_rgb = [0, 0.5, 0.26] 

# font parameters
matplotlib.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['font.size'] = 8

In [ ]:
# load rsc for shared and independent models
with open('rsc_shared_ind_models.csv', "rb") as fp:
    models_rsc = pickle.load(fp)

rsc_shared_model = models_rsc['rsc_shared_model']
rsc_ind_model = models_rsc['rsc_ind_model']

In [ ]:
# plot noise correlation per image
nrow = 2
ncol = 4
fig  = plt.figure(figsize=(6.5,2))
gs_main = gridspec.GridSpec(nrow, ncol, figure=fig, wspace=0.2, hspace=0.2, top=0.98, bottom=0.02, left=0.02, right=0.98)

for i_row in range(0,2):
    
    if i_row == 0:
        rsc = rsc_shared_model
    elif i_row == 1:
        rsc = rsc_ind_model

    for i_col in range(0,4):
        ax = fig.add_subplot(gs_main[i_row,i_col])
        
        if i_col == 3:
            # Compute the mean and standard error of the mean (SEM) for each matrix across the 200 samples
            mean1 = np.nanmean(rsc[:,0,:],axis=1)
            mean2 = np.nanmean(rsc[:,1,:],axis=1)
            sem1 = stats.sem(rsc[:,0,:], axis=1, nan_policy='omit')
            sem2 = stats.sem(rsc[:,1,:], axis=1, nan_policy='omit')

            # Compute the 95% confidence intervals for the mean
            confidence_level = 0.99
            num_nans_per_row_small = np.sum(~np.isnan(rsc[:,0,:]), axis=1) - 1
            num_nans_per_row_large = np.sum(~np.isnan(rsc[:,1,:]), axis=1) - 1

            ci1 = stats.t.interval(confidence_level, num_nans_per_row_small, loc=mean1, scale=sem1)
            ci2 = stats.t.interval(confidence_level, num_nans_per_row_large, loc=mean2, scale=sem2)

            plt.plot(mean1, label="small", color=sz_color[0], lw=1)
            plt.plot(mean2, label="large", color=sz_color[1], lw=1)
            plt.fill_between(range(n_theta), ci1[0], ci1[1], color=sz_color[0], alpha=0.3, lw=0)
            plt.fill_between(range(n_theta), ci2[0], ci2[1], color=sz_color[1], alpha=0.3, lw=0)
            
        else:
            ax.plot(rsc[:,0,simgs_idx[i_col]], linewidth = 1.5, alpha = 0.5)
            ax.plot(rsc[:,1,simgs_idx[i_col]], linewidth = 1.5, alpha = 0.5)

        ax.hlines(0,0,8,linestyles='--',colors=[0.7,0.7,0.7])

        ax.set_ylim([-.2,1])
        ax.set_xlim(0,8)
        ax.set_yticks([0,0.5,1])
        ax.set_yticklabels([0, 0.5, 1])

        if i_row == 0 and i_col == 0:            
            ax.set_ylabel(r'$\mathregular{r_{sc}}$',labelpad=0.25)
            ax.text(-2.5, 0.4, 'shared', ha='center', va='center', rotation=90, color=green_rgb, weight='bold')
        if i_row == 0 and i_col == 3:
            ax.text(2,0.17,f'n = 500\n(natural images)', fontsize = 6, ha='center', va='center', rotation=0, color='k', weight='normal')

        if i_row == 1:
            if i_col == 0:
                ax.set_xlabel(r'$\mathregular{\Delta\Theta}$')
                ax.text(-2.5, 0.4, 'independent', ha='center', va='center', rotation=90, color=purple_rgb, weight='bold')
                plt.legend(['small image', 'large image'], frameon=False, loc=0, prop={'size': 6})
                
            ax.set_xticks([0,2,4,6,8])
            ax.set_xticklabels(['10','30',
                                    '50','70','90'],rotation = 0)  
        else:
            ax.set_xticks([])
        ax.tick_params(axis='both', width = 1, length = 1, pad=3)

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
    plt.savefig('figure3G.pdf', bbox_inches='tight', dpi=300)

In [ ]:
# Create a new Excel writer object
with pd.ExcelWriter('figure3G_data.xlsx') as writer:
    
    # Loop through rows (shared and independent models)
    for i_row in range(2):
        rsc = rsc_shared_model if i_row == 0 else rsc_ind_model
        model_name = 'shared' if i_row == 0 else 'independent'
        
        # Loop through columns (3 individual images + 1 summary)
        for i_col in range(4):
            sheet_name = f'{model_name}_panel{i_col+1}'
            
            if i_col == 3:
                # Summary panel data
                df_summary = pd.DataFrame({
                    'delta_theta': ['10','20','30','40','50','60','70','80','90'],
                    'small_mean': mean1,
                    'small_ci_lower': ci1[0],
                    'small_ci_upper': ci1[1],
                    'large_mean': mean2,
                    'large_ci_lower': ci2[0],
                    'large_ci_upper': ci2[1]
                })
                df_summary.to_excel(writer, sheet_name=sheet_name, index=False)
            else:
                # Individual image data
                df_panel = pd.DataFrame({
                    'delta_theta': ['10','20','30','40','50','60','70','80','90'],
                    'small': rsc[:,0,simgs_idx[i_col]],
                    'large': rsc[:,1,simgs_idx[i_col]]
                })
                df_panel.to_excel(writer, sheet_name=sheet_name, index=False)